# GEDI Data Access 

Authors: Harshini Girish (UAH), Sheyenne Kirkland (UAH), Alex Mandel (DevSeed), Henry Rodman (DevSeed), Zac Deziel (DevSeed)

Date: February 24, 2025

Description: In this notebook, users will learn how to search for GEDI data using `maap-py`, download it, and then open it using `rhdf5`.

## Run This Notebook

To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the ["Getting started with the MAAP"](https://docs.maap-project.org/en/latest/getting_started/getting_started.html) section of our documentation.

Disclaimer: it is highly recommended to run a tutorial within MAAP's ADE, which already includes packages specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors. Users should work within the "R/Python" workspace.

## Additional Resources
- [rhdf5](https://www.bioconductor.org/packages/release/bioc/html/rhdf5.html)
  - The `rhdf5` package page, with installation instructions, documentation, and more.
 
- [NASA's Operational CMR (MAAP Docs)](https://docs.maap-project.org/en/latest/technical_tutorials/search/catalog.html#nasa-s-operational-cmr) 
  - A section in the MAAP Docs offering an overview of resources to search and access NASA's CMR.

- [GEDI02_A v2 Dataset Landing Page](https://lpdaac.usgs.gov/products/gedi02_av002/)
  - Learn more about NASA's GEDI L2A dataset, which is accessed in this notebook.


## Install and Load Required Libraries
Let’s install and load the packages necessary for this tutorial.

Note: as of publishing this notebook, `rhdf5` is not yet within the MAAP workspace. Please run the following code in your terminal before proceeding.

`conda install -c bioconda bioconductor-rhdf5`

In [1]:
library(rhdf5)
library(reticulate)

Let's also invoke the `MAAP` constructor. This will allow us to use the python-based `maap-py` library from R, which will be used to get credentials and conduct a NASA CMR search.

In [2]:
maap_py <- import("maap.maap")
maap <- maap_py$MAAP()

## Get Credentials

Since we will be downloading the GEDI data, we will need temporary credentials for NASA LPDAAC.

In [3]:
credentials = maap$aws$earthdata_s3_credentials(
    "https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials"
)

s3 <- paws::s3(
    credentials = list(
        creds = list(
          access_key_id = credentials["accessKeyId"],
          secret_access_key = credentials["secretAccessKey"],
          session_token = credentials["sessionToken"]
          )),
        region = "us-west-2")

## Collection and Granule Search

Using `maap-py`, we can conduct a collection and granule search for data within NASA's CMR. For this example, we'll use data available within the GEDI L2A collection. For more information on CMR searching in R, see ["Searching for Data in NASA's CMR in R"](https://docs.maap-project.org/en/develop/technical_tutorials/working_with_r/cmr_search_in_r.html). 

In [4]:
# search for a GEDI collection
gedi_collections = maap$searchCollection(
    short_name = "GEDI02_A",
    version = "002",
    cmr_host = "cmr.earthdata.nasa.gov",
    cloud_hosted = "true"
)

# get collection ID for granule search
collection_id = gedi_collections[[1]]["concept-id"]

# search for granules
gedi_granules = maap$searchGranule(
    concept_id = collection_id,
    limit = as.integer(100),
    cmr_host = "cmr.earthdata.nasa.gov"
)

Let's get the S3 URL from the first granule from our granule search.

In [5]:
s3_link <- gedi_granules[[1]]["Granule"]["OnlineAccessURLs"][[1]][1]["URL"]
print(s3_link)

[1] "s3://lp-prod-protected/GEDI02_A.002/GEDI02_A_2019108002012_O01959_01_T03909_02_003_01_V002/GEDI02_A_2019108002012_O01959_01_T03909_02_003_01_V002.h5"


## Download File

Before downloading, lets do some prepping. First we'll create a directory to download our file to. Then from our S3 link, we can get the bucket, key, and a filename.

In [ ]:
# create directory
dir.create("./data")

In [7]:
# create file name for download
filename <- strsplit(s3_link, "/", fixed = TRUE)[[1]] |> tail(n = 1)
filename

# get bucket from file path
bucket = strsplit(s3_link, "/", fixed = TRUE)[[1]] |> head(n = 3)
bucket = bucket[3]
bucket

key <- strsplit(s3_link, "/", fixed = TRUE)[[1]] |> tail(n = 3)
key <- paste(key[1], key[2], key[3], sep = "/")
key

[1] "GEDI02_A_2019108002012_O01959_01_T03909_02_003_01_V002.h5"

[1] "lp-prod-protected"

[1] "GEDI02_A.002/GEDI02_A_2019108002012_O01959_01_T03909_02_003_01_V002/GEDI02_A_2019108002012_O01959_01_T03909_02_003_01_V002.h5"

Now we can download our file.

In [8]:
gedi_file = s3$download_file(Bucket = bucket, Key = key, Filename = paste("./data", filename))

## Access Data

Now that we have our downloaded data, we can use `rhdf5` to open our file for exploration. For the purpose of this notebook, we will use "head" to condense our output.

In [9]:
head(h5ls(paste("./data", filename)))

,group,name,otype,dclass,dim
,<chr>,<chr>,<chr>,<chr>,<chr>
0,/,BEAM0000,H5I_GROUP,,
1,/BEAM0000,ancillary,H5I_GROUP,,
2,/BEAM0000/ancillary,l2a_alg_count,H5I_DATASET,INTEGER,1
3,/BEAM0000,beam,H5I_DATASET,INTEGER,7279
4,/BEAM0000,channel,H5I_DATASET,INTEGER,7279
5,/BEAM0000,degrade_flag,H5I_DATASET,INTEGER,7279
